# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [2]:
# %%capture
# !pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data

In [1]:
import pandas as pd

In [2]:
tweets = pd.read_csv('/Users/jennihawk/Documents/Data Science Projects/Project_NLP/Advanced_NLP_Project/tweets_for_bert.csv')

In [3]:
tweets.shape

(34984, 1)

In [4]:
#turn tweet column into a list of strings
tweet_list = tweets["cleaned"].tolist()

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


In [9]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
#topics, probs = topic_model.fit_transform(my_list_of_documents)
topics, probs = topic_model.fit_transform(tweet_list)

Batches:   0%|          | 0/1094 [00:00<?, ?it/s]

2023-07-13 16:08:55,838 - BERTopic - Transformed documents to Embeddings
2023-07-13 16:11:55,187 - BERTopic - Reduced dimensionality
2023-07-13 16:20:10,346 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [10]:
freq = topic_model.get_topic_info(); freq.head(8)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2347,-1_today_january6thhearings_time_democrats,"[today, january6thhearings, time, democrats, a...",[never forget speaker pelosi trying put end in...
1,0,6000,0_kevin_backed_mitch_mccarthy,"[kevin, backed, mitch, mccarthy, mcconnell, re...",[mitch mcconnell kevin mccarthy knew trump res...
2,1,1611,1_hamill_creating_correct_overthrowing,"[hamill, creating, correct, overthrowing, with...",[maga gop mark hamill correct love country wit...
3,2,1372,2_rudy_hatch_giuliani_meadows,"[rudy, hatch, giuliani, meadows, yet, new, rog...",[new video trump lost yet decided hatch coup h...
4,3,1175,3_demands_deserves_unanimously_history,"[demands, deserves, unanimously, history, test...",[voted unanimously subpoena donald trump testi...
5,4,1106,4_deploy_defense_sec_pathetic,"[deploy, defense, sec, pathetic, acting, perso...",[thing pathetic acting sec defense chris mille...
6,5,1054,5_goaded_author_excuse_summoned,"[goaded, author, excuse, summoned, rioters, at...",[trump author 6th attack trump summoned rioter...
7,6,949,6_rejection_loss_break_accept,"[rejection, loss, break, accept, tried, americ...",[accept loss rejection american people trump t...


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [11]:
freq.shape

(392, 5)

get top 3 most frequent topics

In [12]:
topic_model.get_topic(0)  # Select the most frequent topic

[('kevin', 0.015715863644078114),
 ('backed', 0.01571099085715543),
 ('mitch', 0.015696576008297703),
 ('mccarthy', 0.015684244539004907),
 ('mcconnell', 0.015628809182418532),
 ('responsible', 0.015587749389784912),
 ('called', 0.015549483512407805),
 ('knew', 0.009303137833596231),
 ('trump', 0.004345889958607109),
 ('10h', 0.000126243641746131)]

In [13]:
topic_model.get_topic(1)

[('hamill', 0.037448829877707146),
 ('creating', 0.037448829877707146),
 ('correct', 0.037430114986988575),
 ('overthrowing', 0.03739274220307115),
 ('without', 0.0343921218973107),
 ('love', 0.03434493459965479),
 ('country', 0.03174865598245663),
 ('gop', 0.03140421752748399),
 ('maga', 0.02886555780653262),
 ('violence', 0.02211698494198354)]

In [14]:
topic_model.get_topic(2)

[('rudy', 0.033774867505273774),
 ('hatch', 0.033774867505273774),
 ('giuliani', 0.03358996077962725),
 ('meadows', 0.03323405410548694),
 ('yet', 0.0313421411376195),
 ('new', 0.029042883184728016),
 ('roger', 0.02842249161596429),
 ('stone', 0.0283223465377868),
 ('coup', 0.02789318729955805),
 ('help', 0.027308681793598997)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [15]:
topic_model.topics_[:10]

[30, 4, 4, 100, 0, 10, 175, 2, 112, 1]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [16]:
topic_model.visualize_topics()

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can
be used to understand how confident BERTopic is that certain topics can be found in a document.

To visualize the distributions, we simply call:

In [17]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [18]:
topic_model.visualize_hierarchy(top_n_topics=10)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [19]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [20]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

Output hidden; open in https://colab.research.google.com to view.

### Relationship Between Documents and Topics
- Need to fix error

In [34]:
#reduced_embeddings were calculated beforehand with UMAP - figure out
#Check out this link https://stackoverflow.com/questions/74566683/how-can-i-get-the-coordinate-information-of-model-visualize-topics-function
topic_model.visualize_documents(tweet_list,
                                reduced_embeddings=reduced_embeddings,
                                topics = list(range(30)),
                                custom_labels = True,
                                height=600
)


NameError: ignored

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [23]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [24]:
topic_model.update_topics(tweet_list, n_gram_range=(1, 2))

In [25]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('mccarthy knew', 0.01477087256338062),
 ('responsible called', 0.01477087256338062),
 ('called backed', 0.01477087256338062),
 ('mcconnell kevin', 0.01477087256338062),
 ('backed mitch', 0.014769670857467845),
 ('kevin mccarthy', 0.014749284702443842),
 ('trump responsible', 0.01474497474470467),
 ('kevin', 0.014740667319863813),
 ('knew trump', 0.014738514556578548),
 ('backed', 0.014736362425676164)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [26]:
topic_model.reduce_topics(tweet_list, nr_topics=60)

2023-07-13 16:24:32,980 - BERTopic - Reduced number of topics from 392 to 60


In [27]:
# Access the newly updated topics with:
print(topic_model.topics_)

[19, 3, 3, 2, 1, 0, 17, 1, 0, 2, -1, 1, 2, -1, 0, 0, 19, 2, 3, 1, 0, 3, -1, 1, 16, 50, 20, 2, 1, 50, 51, 4, 0, 19, 0, 6, 36, 13, 6, 13, 0, 0, 1, 0, 1, 6, -1, 0, -1, 6, 5, 19, 7, 7, -1, 1, 15, 6, 4, 0, 13, 1, 5, -1, 1, 0, 0, 4, 0, 0, 0, 16, 19, 0, 1, 3, 18, 1, 29, 18, -1, 2, 16, 19, 20, 1, 37, 9, 2, 19, 7, 0, 19, 0, 1, 0, 1, 19, 19, -1, 0, 18, 19, 0, 6, 0, 6, 19, 19, 1, 1, 1, 1, 19, 0, 14, -1, 11, 1, 19, 0, 16, 19, 0, 37, 6, 19, 1, 12, 0, 19, 15, 2, 0, 19, 26, 16, 0, 14, 0, 13, 2, 0, 0, 19, 37, 2, 48, 19, 2, 26, 0, 0, 19, 0, 0, 5, 0, -1, 19, 0, 0, 1, 19, 0, 7, 0, -1, 0, 8, -1, 0, -1, 7, 0, 1, 0, 17, 0, 2, 1, 0, 0, -1, 2, 2, 0, 51, -1, 0, 2, 0, 7, 1, 22, 1, 20, 0, 37, 0, 2, -1, 48, 0, 0, 0, 0, 0, 32, 13, 16, -1, 1, 0, 0, 48, 0, 2, -1, 0, 2, 1, 0, 1, 0, 1, 0, -1, 0, -1, 0, 1, 3, 0, 0, 1, 1, 0, 7, -1, 8, 7, 0, 0, 0, 2, 1, 0, 48, 0, 2, 0, 4, 0, 0, 1, 4, 1, -1, 0, 0, 4, 8, 5, 0, 2, 2, 1, 0, 15, 1, 1, 6, 0, 0, 1, 0, -1, 0, 5, 39, -1, 7, 1, 0, 1, -1, 7, 47, 0, 1, -1, -1, 5, 1, 0, 1, 9, 7, 1, 1

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "kevin". Then, we extract the most similar topic and check the results:

In [28]:
similar_topics, similarity = topic_model.find_topics("kevin", top_n=5); similar_topics

[1, 26, 0, 6, 58]

#### Question: Not sure why 71?
Not sure what the bool is indicating

In [29]:
topic_model.get_topic(71)

False

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [30]:
# Save model
topic_model.save("my_model")

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [31]:
# Load model
my_model = BERTopic.load("my_model")

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [32]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [33]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
